In [ ]:
%run "Path to the setup.ipynb file"

from transformers import AutoModel, AutoTokenizer
import sys

importing standardlib modules
importing google drive
importing tensorflow/keras modules
importing miscelaneaous modules


In [ ]:
def extract_embeddings(conv_df, emb_path, text_model, tokenizer, max_len):
    """ Extract features for all the conversations and store them in a files
     according to conversations """
    len_utterances = list()
    for n, conv in tqdm(conv_df.groupby("conversation_ID")):
        embeddings = list()
        for text in conv["text"]:
            # tokenize the texts
            text = text.lower()
            text = text.replace("\n", "")
            encoded_input = tokenizer(text,
                                      padding="max_length",
                                      max_length=max_len,
                                      return_tensors='pt')
            # get the model's output
            output = text_model(**encoded_input)

            semantic_feat = output.last_hidden_state

            # transform tensor to numpy
            np_tensor = semantic_feat.detach().numpy()

            # truncate any sequence longer than 40 tokens, applies to < 1%
            embeddings.append(np_tensor[:,:40])
        embeddings = np.array(embeddings)
        np.save(emb_path + f"/conv_{n}_embeddings", embeddings)

    return embeddings, len_utterances

In [ ]:
MODEL_NAME = "bert-large-uncased"

text_model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

emb, lengths = extract_embeddings(conv_df,
                         emb_path="/content/drive/MyDrive/bachelor_thesis/data",
                         text_model=text_model,
                         tokenizer=tokenizer,
                         max_len=40)

  0%|          | 0/1374 [00:00<?, ?it/s]

Average utterance length:  13.867978559365593
Max utterance length:  91
Number of utterances longer than 40 tokens:  136
